In [ ]:
import sys
package_path = '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'
sys.path.append(package_path)

!ls ../input/efficientnet-pytorch/EfficientNet-PyTorch

%cd /kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master
from efficientnet_pytorch import EfficientNet
%cd -

In [ ]:
import pandas as pd
import numpy as np
from fastai.vision.all import *
import pickle
import os

In [ ]:
#Get Image ID path
def get_x(r): return path/'cells'/(r['image_id']+'_'+str(r['cell_id'])+'.jpg')

#Get Image labels
def get_y(r): return r['image_labels'].split('|')

In [ ]:
#Trained efficient model import
export_learner = load_learner('../input/hpaweights/effb5_e2.pkl',cpu=False)

In [ ]:
#Segemented Test Dataset
path = Path('../input/hpa-cell-tiles-test-with-enc-dataset')

In [ ]:
# read dataset
df = pd.read_csv(path/'cell_df.csv')
df.to_csv('cell_df.csv', index=False)

#Test dataloader 
test_dl = export_learner.dls.test_dl(df)

In [ ]:
preds, _ = export_learner.get_preds(dl=test_dl)
preds.shape


In [ ]:
cls_prds = torch.argmax(preds, dim=-1)
len(cls_prds), cls_prds

In [ ]:
sample_submission = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')
sample_submission.head()

In [ ]:
df['cls'] = cls_prds
#assign pred column to class and bbox encorded string
df['pred'] = df[['cls', 'enc']].apply(lambda r: str(r[0]) + ' 1 ' + r[1], axis=1)
df.head()

In [ ]:
subm = df.groupby(['image_id'])['pred'].apply(lambda x: ' '.join(x)).reset_index()
subm.head()

In [ ]:
sub = pd.merge(
    sample_submission,
    subm,
    how="left",
    left_on='ID',
    right_on='image_id',
)
sub.head()

In [ ]:
def isNaN(num):
    return num != num
for i, row in sub.iterrows():
    if isNaN(row['pred']): continue
    sub.PredictionString.loc[i] = row['pred']

In [ ]:
sub = sub[sample_submission.columns]
sub.head()

In [ ]:
sub.to_csv('submission_1.csv', index=False)

In [ ]:
cell_df = pd.read_csv('cell_df.csv')
cell_df.head()
cell_df['cls'] = ''

In [ ]:
threshold = 0.0

for i in range(preds.shape[0]): 
    p = torch.nonzero(preds[i] > threshold).squeeze().numpy().tolist()
    if type(p) != list: p = [p]
    if len(p) == 0: cls = [(preds[i].argmax().item(), preds[i].max().item())]
    else: cls = [(x, preds[i][x].item()) for x in p]
    cell_df['cls'].loc[i] = cls

In [ ]:
def combine(r):
    cls = r[0]
    enc = r[1]
    classes = [str(c[0]) + ' ' + str(c[1]) + ' ' + enc for c in cls]
    return ' '.join(classes)

cell_df['pred'] = cell_df[['cls', 'enc']].apply(combine, axis=1)

In [ ]:
subm = cell_df.groupby(['image_id'])['pred'].apply(lambda x: ' '.join(x)).reset_index()

In [ ]:
sample_submission = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')

In [ ]:
sub = pd.merge(
    sample_submission,
    subm,
    how="left",
    left_on='ID',
    right_on='image_id',
)

In [ ]:
for i, row in sub.iterrows():
    if isNaN(row['pred']): continue
    sub.PredictionString.loc[i] = row['pred']

In [ ]:
sub = sub[sample_submission.columns]


In [ ]:
del export_learner
torch.cuda.empty_cache(),test_dl
#Installing Keras application and tensorlfow efficient Model
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps
#Import tensorlfow  and efficient
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import efficientnet.tfkeras as efn


In [ ]:
#Below function automatically use available gpu and tpu for tensorflow
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy

#Below Function use to read images from dataset for infernce in model
def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img

    def decode_with_labels(path, label):
        return decode(path), label

    return decode_with_labels if with_labels else decode

#below function use to apply  aguments to images to give more variance to model
#We use flip agumentation
def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img

    def augment_with_labels(img, label):
        return augment(img), label

    return augment_with_labels if with_labels else augment

#Building utility function for testing dataset with batch size =32
def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)

    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)

    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)

    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)

    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)

    return dset

In [ ]:
COMPETITION_NAME = "hpa-single-cell-image-classification"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16

IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600)

load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
sub_df = sub.copy()

In [ ]:
sub_df = sub_df.drop(sub_df.columns[1:],axis=1)
for i in range(19):
    sub_df[f'{i}'] = pd.Series(np.zeros(sub_df.shape[0]))
test_paths = load_dir + "/test/" + sub_df['ID'] + '_green.png'
label_cols = sub_df.columns[1:]
test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[7], IMSIZE[7]))
dtest = build_dataset(
        test_paths, bsize=BATCH_SIZE, repeat=False, 
        shuffle=False, augment=False, cache=False,
        decode_fn=test_decoder
    )

In [ ]:
with strategy.scope():
    model = tf.keras.models.load_model(
        '../input/hpa-classification-efnb7-train/model_green.h5'
    )

In [ ]:
model.summary()
sub_df[label_cols] = model.predict(dtest, verbose=1)

In [ ]:
sub = pd.merge(sub, sub_df, on = 'ID', how = 'left')

In [ ]:
for i in range(sub.shape[0]):
    if sub.loc[i,'PredictionString'] == '0 1 eNoLCAgIMAEABJkBdQ==':
        continue
    a = sub.loc[i,'PredictionString']
    b = a.split()
    for j in range(int(len(a.split())/3)):
        for k in range(19):
            if int(b[0 + 3 * j]) == k:

                c = b[0 + 3 * j + 1]               
                b[0 + 3 * j + 1] = str(sub.loc[i,f'{k}'] * 0.6 + float(c) * 0.4)# * 0.9 + float(c) * 0.1

    sub.loc[i,'PredictionString'] = ' '.join(b)

In [ ]:
sub = sub[['ID','ImageWidth','ImageHeight','PredictionString']]

In [ ]:
sub.to_csv('submission.csv', index=False)